Run docker container as:

```
chmod o+wrx /var/run/docker.sock

JUPYTERLAB_VERSION=1.1.0
REPO_URL=palmoreck/jupyterlab_geopython_for_conabio_cluster

docker run -dit -e LOCAL_USER_ID=$(id -u epalacios) --rm -v /var/run/docker.sock:/var/run/docker.sock -v /LUSTRE/MADMEX/:/LUSTRE/MADMEX --name jupyterlab_geopython -p 8888:8888 -d $REPO_URL:$JUPYTERLAB_VERSION bash


docker exec -u=geonode_user -d jupyterlab_geopython bash -c "/usr/local/bin/jupyter lab --ip=0.0.0.0 --no-browser"
```


# Large size Vectors

In [1]:
import unicodedata


from pyproj import Proj

import fiona
from fiona.crs import to_string
from rasterio.warp import transform_geom
from rasterio.crs import CRS as CRS_rio

In [2]:
direc = '/datos/products/landcoverchange/landsat/mosaics/2017_2018'

In [3]:
input_file = direc + '/madmex_landsat_changes_2017-2018.shp'
out_filename = direc + '/madmex_landsat_changes_2017-2018_wgs84_using_fiona.shp'
layer = 'madmex_landsat_changes_2017-2018_wgs84_using_fiona'

dst_crs = '4326'
name_attribute1='t1_dsc_31'
name_attribute2='t2_dsc_31'
name_attribute3='t1_dsc_17'
name_attribute4='t2_dsc_17'
name_attribute5='cmb_dsc_31'
name_attribute6='cmb_dsc_17'

In [4]:
list_name_attributes = [name_attribute1, name_attribute2,
                        name_attribute3, name_attribute4,
                        name_attribute5, name_attribute6]

In [5]:
def reproj_geom(geom, source_crs, destiny_crs):
    return transform_geom(CRS_rio.from_proj4(source_crs), CRS_rio.from_epsg(destiny_crs),
                          geom)

In [6]:
def normalize_name_classes(string):
    return unicodedata.normalize('NFKD', string).encode('ASCII','ignore').decode('utf-8')

In [7]:
def wrapper_reproj_and_normalize_vector(feature_collection, source_crs, destiny_crs):
    dst = fiona.open(out_filename, 'w',
                     driver = 'ESRI Shapefile',
                     layer = layer,
                     crs = CRS_rio.from_epsg(dst_crs).to_proj4(),
                     schema = fc_schema)
    dst.close()
    with fiona.open(out_filename, 'a',
                    driver = 'ESRI Shapefile',
                    layer = layer,
                    crs = CRS_rio.from_epsg(destiny_crs).to_proj4(),
                    schema = fc_schema) as dst:
        for feature in feature_collection:
            feature['geometry'] = reproj_geom(feature['geometry'],
                                              source_crs,
                                              destiny_crs)
            feature['properties'][name_attribute1] = normalize_name_classes(feature['properties'][name_attribute1])
            feature['properties'][name_attribute2] = normalize_name_classes(feature['properties'][name_attribute2])
            feature['properties'][name_attribute3] = normalize_name_classes(feature['properties'][name_attribute3])
            feature['properties'][name_attribute4] = normalize_name_classes(feature['properties'][name_attribute4])
            feature['properties'][name_attribute5] = normalize_name_classes(feature['properties'][name_attribute5])
            feature['properties'][name_attribute6] = normalize_name_classes(feature['properties'][name_attribute6])
            dst.write(feature)

In [ ]:
with fiona.open(input_file) as src:
    #fc=[k for k in src]
    fc = (feature for feature in src if feature['properties'][name_attribute1] is not None and
                                        feature['properties'][name_attribute2] is not None and
                                        feature['properties'][name_attribute3] is not None and
                                        feature['properties'][name_attribute4] is not None and
                                        feature['properties'][name_attribute5] is not None and
                                        feature['properties'][name_attribute6] is not None
         )
    src_crs = to_string(src.crs)
    proj_crs = Proj(src_crs)
    fc_schema = src.schema
    if not proj_crs.crs.is_geographic:
        wrapper_reproj_and_normalize_vector(fc, src_crs, dst_crs)
    else:
        wrapper_normalize_vector(fc, src_crs, dst_crs)
        

# Insert in geonode

**Make sure stack deployment of geonode is up and running**

In [ ]:
%%bash
sudo apt install -yq postgis postgresql-10-postgis-scripts

**1) Create `~/.pgpass` with contents: `hostname:port:database:username:password` and `chmod 0600 ~/.pass`**

In [ ]:
%%bash
shp2pgsql /datos/products/landcoverchange/landsat/mosaics/2017_2018/madmex_landsat_changes_2017-2018_wgs84_using_fiona.shp madmex_landsat_2017-2018_lcc public.madmex_landsat_changes_2017-2018_wgs84.shp | psql -q -h geonode.conabio.gob.mx -d geonode_data -U geonode

**2) Add to geoserver from geonode_data database**

**3) Update layers inside docker container**

**Create id_rsa.pub with `ssh-keygen` and add it to `/home/epalacios/.ssh/authorized_keys`**

In [ ]:
%%bash
#next are not working:
ssh epalacios@172.17.0.1 docker exec spcgeonode_django_1 /bin/bash -c "DJANGO_SETTINGS_MODULE=geonode.local_settings python manage.py updatelayers -s geonode_data -w geonode -f madmex_sentinel2_chihuahua_2017_2018_lcc"
ssh epalacios@172.17.0.1 docker exec -e file=madmex_landsat_2017-2018_lcc spcgeonode_django_1 /bin/bash -c "bash /spcgeonode/script_updatelayers.sh $file"

```
pip3 install docker
```

In [1]:
from docker import APIClient

In [2]:
c = APIClient(base_url='unix://var/run/docker.sock')

In [3]:
c

In [100]:
#c.info()

In [93]:
cmd = 'python manage.py importlayers ' + \
      '-v 3 -i -o -n madmex_sentinel2_2017_31_tiled -t madmex_sentinel2_2017_31_tiled ' + \
      '-a "Sentinel2 MAD-Mex lc" ' + \
      '-k "MAD-Mex, Sentinel2, GeoTIFF, WCS" ' + \
      '-r "Mexico, North America, Latin America" ' + \
      '/LUSTRE/MADMEX/products/landcover/sentinel2/2017/31/madmex_sentinel2_2017_31_wgs84_tiled.tif ' + \
      '--settings=geonode.local_settings'

In [94]:
cmd

'python manage.py importlayers -v 3 -i -o -n madmex_sentinel2_2017_31_tiled -t madmex_sentinel2_2017_31_tiled -a "Sentinel2 MAD-Mex lc" -k "MAD-Mex, Sentinel2, GeoTIFF, WCS" /LUSTRE/MADMEX/products/landcover/sentinel2/2017/31/madmex_sentinel2_2017_31_wgs84_tiled.tif --settings=geonode.local_settings'

In [95]:
ex = c.exec_create(container = 'spcgeonode_django_1', 
                   cmd = cmd)

In [96]:
ex_start = c.exec_start(exec_id=ex, stream=True)

In [99]:
#for val in ex_start:
#    print(val)

In [43]:
ex = c.exec_create(container = 'spcgeonode_django_1', 
                   cmd = 'bash /spcgeonode/script_updatelayers.sh ' + 'madmex_landsat_2017-2018_lcc')

In [ ]:
#alternative:
#cmd = 'python manage.py updatelayers -s geonode_data -w geonode -f ' + \
#      'madmex_landsat_2017-2018_lcc'
#ex = c.exec_create(container = 'spcgeonode_django_1', 
#                   cmd = cmd)

In [44]:
ex_start = c.exec_start(exec_id=ex, stream=True)

In [98]:
#for val in ex_start:
#    print(val)

## Small to Medium size vectors

In [1]:
import geopandas as gpd

## Landsat

In [2]:
direc = '/datos/products/landcoverchange/sentinel2/2017_2018/indi50k/estados/AGUASCALIENTES'

In [6]:
input_file = direc + '/AGUASCALIENTES_merge.shp'
out_filename = direc + '/AGUASCALIENTES_merge_wgs84_using_fiona.shp'
layer = 'AGUASCALIENTES_merge_wgs84_using_fiona'

In [7]:
gdf = gpd.read_file(input_file)

In [ ]:
crs_src = gdf.crs

In [ ]:
crs_dst = "EPSG:4326"

In [ ]:
gdf_reproj = gpd.to_crs(crs_dst)